## 1. Setup StockX

In [1]:
# In your Jupyter notebook (TEST_Daily_Analysis.ipynb)
import sys
from pathlib import Path

# Add the project root directory to Python path
ROOT_DIR = Path.cwd().parent  # Goes up one level from 'scanner' to 'stockbot4'
if str(ROOT_DIR) not in sys.path:
    sys.path.append(str(ROOT_DIR))


from ib_insync import *
util.startLoop()
import stock

ib = IB()
ib.connect('127.0.0.1', 7496, clientId=12)
sx = stock.StockX(ib, 'MAT')
spy = stock.StockX(ib, 'SPY')

## 2. Fundamentals

In [2]:

sx.req_fundamentals(max_days_old=10)  # gets the fundamentals from the IB API
sx.sector_ETF_is_allowed(['XLY', 'XLK', 'XLC'])


Using cached data for MAT (age: 2025-02-05 18:26:37)


True

## 3. Setup Frames

In [3]:
# if sx.sector_ETF_is_allowed():
sx.setup_all_frames('ohlcv', force_download=False)
spy.setup_all_frames('ohlcv', force_download=False)
display(sx.get_status_df())
print(sx.frames.keys())


Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\MAT_1_day.csv
Stored data: 275 rows of data
Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\MAT_1_day.csv
Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\MAT_1_hour.csv
Stored data: 253 rows of data
Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\MAT_1_hour.csv
Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\MAT_1_min.csv
Stored data: 3496 rows of data
Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\MAT_1_min.csv
Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\SPY_1_day.csv
Stored data: 285 rows of data
Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\SPY_1_day.csv
Loading data from C:\Users\sidsu\anaconda3\envs\SB4\s

""


dict_keys(['1 day', '1 hour', '5 mins'])


## 4. Import Market Data 

In [4]:
sx.import_all_market_data(spy)
sx.frames['1 hour'].data

,open,high,low,close,volume,SPY_high,SPY_volume,SPY_low,SPY_open,SPY_close
date,,,,,,,,,,
2025-01-13 06:00:00,18.44,18.44,18.44,18.44,1,576.23,256,575.50,575.50,575.97
2025-01-13 07:00:00,18.44,18.44,18.44,18.44,0,577.48,3007,575.30,576.02,576.94
2025-01-13 08:00:00,18.50,18.50,18.50,18.50,4,578.77,2842,575.31,575.80,576.78
2025-01-13 09:00:00,18.56,18.62,18.45,18.55,862,578.32,49344,575.35,576.78,577.87
2025-01-13 10:00:00,18.54,18.67,18.40,18.52,1389,579.18,45061,575.94,577.88,577.90
...,...,...,...,...,...,...,...,...,...,...
2025-02-05 09:00:00,20.33,21.35,20.26,21.21,24375,601.09,24227,599.47,600.00,600.12
2025-02-05 10:00:00,21.23,21.33,20.18,20.21,42410,601.10,34701,598.58,600.12,600.59
2025-02-05 11:00:00,20.21,20.95,20.18,20.93,19551,602.88,30361,600.13,600.57,602.65


In [5]:
# sx.run_daily_frame(lookback=300)

In [5]:
import pandas as pd
import strategies.preset_strats as ps
import strategies.signals as sig
import strategies.ta as ta

def run_day_frame(sx, ls, lookBack, atrSpan, scoreRow, validationRow):
    f = sx.frames['1 day']
    # TA setup 
    pointsSpan = 10
    ps.ma_ta(f, [50, 150, 200])
    f.add_ta(ta.ATR(span=atrSpan), {'dash': 'solid', 'color': 'cyan', 'width': 1}, row=3, chart_type='')
    f.add_ta(ta.HPLP(hi_col='high', lo_col='low', span=3), [{'color': 'green', 'size': 3}, {'color': 'red', 'size': 10}], chart_type = 'points')
    f.add_ta(ta.HPLP(hi_col='high', lo_col='low', span=pointsSpan), [{'color': 'green', 'size': 10}, {'color': 'red', 'size': 4}], chart_type = 'points')
    f.add_ta(ta.SupResAllRows(hi_point_col=f'HP_hi_{pointsSpan}', lo_point_col=f'LP_lo_{pointsSpan}', atr_col=f'ATR_{atrSpan}', tolerance=1, rowsToUpdate=10),
        [{'dash': 'solid', 'color': 'green', 'fillcolour': "rgba(0, 255, 0, 0.1)", 'width': 1}, # support # green = rgba(0, 255, 0, 0.1)
        {'dash': 'solid', 'color': 'red', 'fillcolour': "rgba(255, 0, 0, 0.1)", 'width': 1}], # resistance # red = rgba(255, 0, 0, 0.1)
        chart_type = 'support_resistance')
    f.add_ta(sig.RoomToMove(ls=ls, tgetCol='Res_1_Lower', atrCol=f'ATR_{atrSpan}', unlimitedVal=10, normRange=(0,30), lookBack=lookBack), {'dash': 'solid', 'color': 'yellow', 'width': 2}, chart_type='line', row=scoreRow)
    f.add_ta(sig.GapsSize( atrCol=f'ATR_{atrSpan}', normRange=(0,100), lookBack=lookBack), {'dash': 'solid', 'color': 'yellow', 'width': 2}, chart_type='line', row=scoreRow)
    f.add_ta(sig.PctDiff(metricCol1='close', metricCol2='MA_cl_50', lookBack=lookBack), {'dash': 'solid', 'color': 'yellow', 'width': 1}, chart_type='line', row=scoreRow)
    f.add_ta(ta.RSATRMA(comparisonPrefix='SPY', ma=14, atr=20), 
             [{'dash': 'solid', 'color': 'yellow', 'width': 1},
              {'dash': 'solid', 'color': 'cyan', 'width': 1}], 
             chart_type='line', row=scoreRow)

    # -- Validations --  
    if ls == 'LONG':
        # above 50 MA
        f.add_ta(sig.Validate(f, val1='close',                 operator='>',  val2='MA_cl_50', normRange=(0,1), lookBack=lookBack), {'dash': 'solid', 'color': 'lime', 'width': 1}, chart_type='line', row=validationRow)
        f.add_ta(sig.Validate(f, val1='PctDiff_MA_cl_50',      operator='>',  val2=0,          normRange=(0,1), lookBack=lookBack), {'dash': 'solid', 'color': 'lime', 'width': 1}, chart_type='line', row=validationRow)
        f.add_ta(sig.Validate(f, val1='close',                 operator='^p', val2='HP_hi_3',  normRange=(0,1), lookBack=lookBack), {'dash': 'solid', 'color': 'lime', 'width': 1}, chart_type='line', row=validationRow)
        f.add_ta(sig.Validate(f, val1='close',                 operator='^p', val2='HP_hi_10', normRange=(0,1), lookBack=lookBack), {'dash': 'solid', 'color': 'cyan', 'width': 1}, chart_type='line', row=validationRow)
        f.add_ta(sig.Validate(f, val1='GapSz',                 operator='><', val2=(2,10),     normRange=(0,1), lookBack=lookBack), {'dash': 'solid', 'color': 'lime', 'width': 1}, chart_type='line', row=validationRow)
        f.add_ta(sig.Validate(f, val1='RTM_L_Res_1_Lower',  operator='>',  val2=2,          normRange=(0,1), lookBack=lookBack), {'dash': 'solid', 'color': 'lime', 'width': 1}, chart_type='line', row=validationRow)
        f.add_ta(sig.Validate(f, val1='RS_SPY_20',             operator='>',  val2=2,          normRange=(0,1), lookBack=lookBack), {'dash': 'solid', 'color': 'lime', 'width': 1}, chart_type='line', row=validationRow)

    # --- Final Score ---
    l_or_s = ls[0]
    cols = list(f.data.columns[-7:])
    f.add_ta(sig.Score(name=f'{l_or_s}_VALIDATION', cols=cols, scoreType='mean', weight=1, lookBack=lookBack), {'dash': 'solid', 'color': 'magenta', 'width': 3}, chart_type='line', row=5)

run_day_frame(sx, 'LONG', 25, 14, 3, 4)
sx.frames['1 day'].plot()

KeyError: 'SigL_PctDiff_MA_cl_50'

In [8]:
cols = list(sx.frames['1 day'].data.columns[-7:])

sx.frames['1 day'].data[cols]

,AB_0_be_SigL_PctDiff_MA_cl_50,BRK_ab_HP_hi_3,BRK_ab_HP_hi_10,BTWN_2_10_GapSz,AB_2_be_SigL_RTM_Res_1_Lower,AB_2_be_RS_SPY_20,Score_L_VALIDATION
date,,,,,,,
2024-02-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-02-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-02-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-02-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-02-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2025-01-30,100.0,0.0,100.0,0.0,0.0,0.0,100.0
2025-01-31,100.0,0.0,0.0,0.0,0.0,0.0,100.0
2025-02-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:

cols = [c for c in sx.frames['1 day'].data.columns if 'RS_' in c]

sx.frames['1 day'].data[cols]

,RS_SPY__20,RS_MA_SPY__20_14
date,,
2024-02-06,NaN,NaN
2024-02-07,NaN,NaN
2024-02-08,NaN,NaN
2024-02-09,NaN,NaN
2024-02-12,NaN,NaN
...,...,...
2025-01-30,-0.170251,-0.113427
2025-01-31,0.044089,-0.190548
2025-02-03,0.343406,-0.188937


In [ ]:
sx.frames['1 day'].plot()
sx.frames['1 day'].data['PBX_ALL_Scores'].iat[-1]
sx.get_score_status_by_item('1 day', 'ohlcv')

0.0

In [ ]:
sx.run_intraday_frames(plot=True)

Ran intraday frames: ['1 hour', '5 mins', '2 mins']
Plotting 1 hour


Plotting 5 mins


Plotting 2 mins


In [ ]:
# pd max rows
import pandas as pd
pd.set_option('display.max_rows', 500)
sx.frames['1 hour'].data

,open,high,low,close,volume,ATR_50,HP_hi_10,LP_lo_10,Res_1,Res_1_Upper,...,Res_2_Lower,Sup_1,Sup_1_Upper,Sup_1_Lower,Sup_2,Sup_2_Upper,Sup_2_Lower,MA_cl_50,MA_cl_150,MA_cl_200
date,,,,,,,,,,,,,,,,,,,,,
2024-12-09 09:00:00,397.80,404.80,393.80,398.27,150935,11.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-12-09 10:00:00,398.38,399.00,380.90,383.77,174880,14.550000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-12-09 11:00:00,383.64,384.50,378.01,382.70,91461,11.863333,NaN,378.01,NaN,NaN,...,NaN,378.01,378.01,378.01,NaN,NaN,NaN,NaN,NaN,NaN
2024-12-09 12:00:00,382.68,384.97,380.75,382.64,55407,9.952500,NaN,NaN,NaN,NaN,...,NaN,378.01,378.01,378.01,NaN,NaN,NaN,NaN,NaN,NaN
2024-12-09 13:00:00,382.48,387.84,382.03,386.87,55369,9.124000,NaN,NaN,NaN,NaN,...,NaN,378.01,378.01,378.01,NaN,NaN,NaN,NaN,NaN,NaN
2024-12-09 14:00:00,386.92,387.86,383.24,384.96,46993,8.373333,NaN,NaN,NaN,NaN,...,NaN,378.01,378.01,378.01,NaN,NaN,NaN,NaN,NaN,NaN
2024-12-09 15:00:00,384.90,390.00,382.49,389.88,79739,8.250000,NaN,NaN,NaN,NaN,...,NaN,378.01,378.01,378.01,NaN,NaN,NaN,NaN,NaN,NaN
2024-12-10 09:00:00,392.68,403.17,390.85,400.52,132351,8.758750,NaN,NaN,NaN,NaN,...,NaN,378.01,378.01,378.01,NaN,NaN,NaN,NaN,NaN,NaN
2024-12-10 10:00:00,400.50,407.82,400.34,401.53,149416,8.616667,NaN,NaN,NaN,NaN,...,NaN,378.01,378.01,378.01,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:

#! Draft example of how to use the StockX class

# import stock
# from strategies import ta

# sx = stock.StockXDaily(ib, "TSLA")

# # getting the stock data
# allowed_etfs = ['XLK', 'XLY', 'XLI']
# sx.req_fundamentals(max_days_old=1) # gets the fundamentals from the IB API
# sx.validate_fundamental('primary_etf',      'isin', allowed_etfs,    description='Stocks primary secteor ETF is allowed')
# sx.validate_fundamental('secondary_etf',    'isin', allowed_etfs,    description='Stocks primary secteor ETF is allowed')
# sx.validate_fundamental('market_cap',       '>=', 300,               description='Market cap is greater than 300M') 
# sx.validate_fundamental('volume_10day_avg', '>=', 0.3,               description='Volume is greater than 300k')
# display(sx.validation_fundamentals_report(asDF=True, save_image=True)) # returns a dictionary with the validation results
# sx.validation_fundamentals_has_passed(maxFails=0) # returns boolean if all validations are True
# sx.get_funadmentals_validation_results(allowed_etfs)

In [ ]:
 # gets the ohlcv data from the IB APIs
sx.req_ohlcv()
# price
sx.validate_TA(ta.ColVal('close'),           ta.AboveBelow('CV_close', 'above', 1),                                                                  description='close price is above 1')
sx.validate_TA(ta.MA('close', 200),          ta.AboveBelow('close', 'above', 'MA_cl_200',), {'dash': 'solid', 'color': 'cyan', 'width': 3},   row=1, description='close price is above 200 MA') 
sx.validate_TA(ta.MA('close', 150),          ta.AboveBelow('close', 'above', 'MA_cl_150',), {'dash': 'solid', 'color': 'pink', 'width': 2},   row=1, description='close price is above 150 MA') 
sx.validate_TA(ta.MA('close', 50),           ta.Breaks    ('close', 'above', 'MA_cl_50'),   {'dash': 'solid', 'color': 'purple', 'width': 2}, row=1, description='close price breaks above 50 MA')
sx.validate_TA(ta.PctChange('MA_cl_50', 1),  ta.AboveBelow('PCT_MA_cl_50_1', 'above', 0) ,                                                           description='pct change of 50 MA is above 0')
sx.validate_TA(ta.PctChange('close', 1),     ta.AboveBelow('PCT_close_1', 'above', 4) ,                                                              description='pct change of close is above 4 (4% Gap)')
# volume
sx.validate_TA(ta.ColVal('volume'),          ta.AboveBelow('CV_volume','above', 50_000),                                                             description='volume is above 50k')
sx.validate_TA(ta.MA('volume', 10),          ta.Breaks('volume','above', 'MA_vo_10'), {'dash': 'solid', 'color': 'pink', 'width': 1}, row=2,         description='volume breaks above 10 MA')
sx.validate_TA(ta.VolDev('volume', 10),      ta.AboveBelow('VDEV_10', 'above', 80), {'dash': 'solid', 'color': 'pink', 'width': 1}, row=3,           description='volume is above 80% of 10 MA Deviation')

df = sx.validation_TA_report(asDF=True, save_image=True) # returns a dictionary with the validation results
print(sx.validation_TA_has_passed()) # returns boolean if all validations are True
display(df)
sx.get_TA_validation_results()



AttributeError: 'StockX' object has no attribute 'req_ohlcv'

 # StockX Score

In [ ]:
import stock
from strategies import ta 
from strategies import signals as sig 


sx = stock.StockXDaily(ib, "TSLA")

sx.req_ohlcv()
sx.add_TA(ta.HPLP(span=10), {'dot': 'solid', 'color': 'cyan', 'width': 3},   row=1)
sx.add_TA(ta.ATR(span=14))
sx.score_TA(sig.GappedPivots(hpCol='HP_hi_10', lpCol='LP_lo_10', span=20),  sig.Score('SigL_GPIV', (0,100), weight=1), {'dash': 'solid', 'color': 'cyan', 'width': 3},   row=1, description='close price is above 200 MA')
sx.score_TA(sig.GappedBarQuality(atrCol='ATR_10', atrMultiple=2),           sig.Score('GBQ_2', (0,100), weight=1), {'dash': 'solid', 'color': 'cyan', 'width': 3},   row=1, description='close price is above 200 MA')
sx.score_TA(sig.GappedBarCount(),                                           sig.Score('GBC', (0,100), weight=1), {'dash': 'solid', 'color': 'cyan', 'width': 3},   row=1, description='close price is above 200 MA')
sx.score_TA(sig.GappedPastPivot(atrCol='ATR_10'),                           sig.Score('GPP', (0,100), weight=1), {'dash': 'solid', 'color': 'cyan', 'width': 3},   row=1, description='close price is above 200 MA')
 
sx.score_group(sig.ScoreGroup('Retest', ['Olap', 'Pullback', 'Breakout'], [50, 30, 20], [True, True, True], 'mean', weight=1), description='Retest group')

Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\TSLA_1_day.csv
Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\TSLA_1_day.csv
Updating signals for GPIV...


KeyError: 'GPIV'

In [ ]:
sx.frame.data.tail(50)

,open,high,low,close,volume,HP_hi_10,LP_lo_10,ATR_14
date,,,,,,,,
2024-10-08,241.50,246.21,240.05,243.84,425761,NaN,NaN,6.160000
2024-10-09,243.92,247.43,239.51,241.56,532290,NaN,NaN,7.040000
2024-10-10,242.40,244.30,232.34,239.68,666901,NaN,NaN,8.680000
2024-10-11,230.10,238.77,214.38,217.97,1076253,NaN,NaN,12.607500
2024-10-14,220.70,222.00,213.74,218.90,655061,NaN,NaN,11.738000
2024-10-15,218.69,224.26,216.01,219.02,496586,NaN,NaN,11.156667
2024-10-16,219.49,222.82,218.93,220.95,404512,NaN,NaN,10.118571
2024-10-17,222.00,223.60,217.94,221.24,383329,NaN,NaN,9.561250
2024-10-18,221.12,222.28,218.71,220.14,356148,NaN,NaN,8.895556


In [ ]:
import pandas as pd
import stock
from strategies import ta

def update_scan_results(scan_results_df, allowed_etfs, ib):
    for s in scan_results_df['symbol']:
        sx = stock.StockXDaily(ib, s)
        sx.req_fundamentals(max_days_old=1)  # gets the fundamentals from the IB API
        sx.req_ohlcv()
        fund_results = sx.get_funadmentals_validation_results(allowed_etfs)
        ta_results   = sx.get_TA_validation_results()

        # Find the index of the row with the symbol
        index = scan_results_df[scan_results_df['symbol'] == s].index[0]

        # Update the row with the validation results
        for key, value in fund_results.items():
            scan_results_df.loc[index, key] = value
        
        # Update the row with the validation results
        for key, value in ta_results.items():
            scan_results_df.loc[index, key] = value

    return scan_results_df

# Example usage
scan_results_df = pd.DataFrame({
    'rank': [1, 2, 3],
    'symbol': ['TSLA', 'AAPL', 'MSFT'],
})



updated_scan_results_df = update_scan_results(scan_results_df, ['XLK', 'XLY', 'XLI'], ib)
updated_scan_results_df

Using cached data for TSLA (age: 2024-12-22 09:34:58)
Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\TSLA_1_day.csv
Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\TSLA_1_day.csv
Using cached data for AAPL (age: 2024-12-22 09:44:08)
Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\AAPL_1_day.csv
Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\AAPL_1_day.csv
Using cached data for MSFT (age: 2024-12-22 09:44:10)
Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\MSFT_1_day.csv
Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\MSFT_1_day.csv


,rank,symbol,Sector1 Valid,Market Cap > 300M,Vol 10DayMA > 300K,Fundamentals Passed,Close > $1,Above 200MA,Above 150MA,Breaks Above 50MA,50MA Slope > 0,Gap Up > 4%,Volume > 50K,Volume Above 10MA,Volume Dev > 80%,TA Passed
0,1,TSLA,True,True,True,1.0,True,True,True,False,True,False,True,False,False,0.56
1,2,AAPL,True,True,True,1.0,True,True,True,False,True,False,True,False,False,0.56
2,3,MSFT,True,True,True,1.0,True,True,True,False,True,False,True,False,False,0.56


In [ ]:
ib.disconnect()

In [ ]:
sx.frame.setup_chart()
sx.frame.plot() # plots the stock data  
sx.save_chart() # saves the chart to a file
sx.save_zoomed_chart(show=True) # shows the chart in the notebook


Chart saved as TSLA.png
Zoomed chart saved as TSLA_zoomed.png


In [ ]:
sx.email_report()

Email sent to for TSLA


In [ ]:

# scoring the stock
sx.score_fundamantal('pe_ratio', 10, 20) # returns a score between 0 and 100
sx.score_fundamantal('consensus_recommendation', 1, 20) # returns a score between 0 and 100
sx.score_fundamantal_report() # returns a dictionary with the scoring results
sx.score_fundamantal_has_passed(minScore=20, minAvScore=50) # returns boolean if all scores are True

# scoring the technical analysis
sx.score_TA(ta.MansfieldRSI('close', 'SPY', 200),  0, 2) # returns a score between 0 and 100
sx.score_TA(ta.MansfieldRSI('close', 'XLK', 200),  0, 2) # returns a score between 0 and 100
sx.score_TA(ta.PivotsBroken('HP1'),  0, 2) # returns a score between 0 and 100
sx.score_TA(ta.RoomToMove('Res1'),  0, 2) # returns a score between 0 and 100
sx.score_TA(ta.VolAcc(0.0001),  0, 100) # returns a score between 0 and 100
sx.score_TA_report() # returns a dictionary with the scoring results
sx.score_TA_has_passed(minScore=20, minAvScore=50) # returns boolean if all scores are True

AttributeError: 'StockXDaily' object has no attribute 'score_fundamantal'

In [ ]:
import stock
sx = stock.StockXDaily(ib, "TSLA")
sx.req_fundamentals()

{'industry': 'Consumer, Cyclical',
 'category': 'Auto Manufacturers',
 'subcategory': 'Auto-Cars/Light Trucks',
 'primary_etf': ('XLY', 0.56),
 'secondary_etf': ('XLI', 0.44),
 'currency': 'USD',
 'longName': 'TESLA INC',
 'timeZoneId': 'US/Eastern',
 'tradingHours': ('0400', '2000'),
 'liquidHours': ('0930', '1600'),
 'current_price': 389.79,
 'high_52week': 404.8,
 'low_52week': 138.8025,
 'pricing_date': '2024-12-09T00:00:00',
 'volume_10day_avg': 69.95339,
 'enterprise_value': 1261250.0,
 'market_cap': 1287202.0,
 'revenue_ttm': 97150.0,
 'ebitda': 13338.0,
 'net_income_ttm': 12743.0,
 'eps_ttm': 3.64717,
 'revenue_per_share': 27.83867,
 'book_value_per_share': 21.80574,
 'cash_per_share': 10.49205,
 'cash_flow_per_share': 5.12014,
 'dividend_per_share': 0.0,
 'gross_margin': 18.22851,
 'operating_margin': 0.0,
 'net_profit_margin': 0.0,
 'revenue_growth_rate': 0.0,
 'eps_growth_rate': 0.0,
 'pe_ratio': 109.9455,
 'price_to_book': 18.3892,
 'price_to_sales': 0.0,
 'employee_count':

In [ ]:

sx.req_fundamentals()
sx.fundamentals.__dict__

In [ ]:
sx.validate_fundamental

In [ ]:
tsla.get_daily_data()

Loading data from data/historical_data_store/TSLA_1_day.csv
Stored data: 250 rows of data
Loading data from data/historical_data_store/TSLA_1_day.csv


,open,high,low,close,volume
date,,,,,
2023-12-06,240.61,246.57,238.85,240.37,1080540
2023-12-07,240.80,244.08,236.98,242.91,916952
2023-12-08,242.00,245.27,239.27,243.23,878583
2023-12-11,242.00,245.00,237.45,239.55,819699
2023-12-12,240.25,241.22,233.87,236.48,785221
...,...,...,...,...,...
2024-11-26,338.00,346.96,335.21,340.17,470824
2024-11-27,339.76,342.60,326.59,334.40,441879
2024-11-29,336.00,348.50,334.65,345.44,272292


In [ ]:
tsla.set_default_daily_ta()
tsla.frame.data

,open,high,low,close,volume,MA_cl_200,MA_cl_50,MA_vo_10,ACC_close,VDEV_10,TDUR_MA_cl_50
date,,,,,,,,,,,
2023-12-06,240.61,246.57,238.85,240.37,1080540,NaN,NaN,NaN,NaN,NaN,1
2023-12-07,240.80,244.08,236.98,242.91,916952,NaN,NaN,NaN,NaN,NaN,1
2023-12-08,242.00,245.27,239.27,243.23,878583,NaN,NaN,NaN,NaN,NaN,1
2023-12-11,242.00,245.00,237.45,239.55,819699,NaN,NaN,NaN,NaN,NaN,1
2023-12-12,240.25,241.22,233.87,236.48,785221,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...
2024-11-26,338.00,346.96,335.21,340.17,470824,213.43710,269.0762,723829.0,65.76,-34.953698,45
2024-11-27,339.76,342.60,326.59,334.40,441879,214.18910,271.1760,671255.0,48.25,-34.171217,46
2024-11-29,336.00,348.50,334.65,345.44,272292,214.96490,273.2024,605475.7,21.40,-55.028418,47


In [ ]:

tsla.compute_daily_filters_with_scores()


,BRK_UP_MA,BRK_UP_MA,ABV_close_MA_cl_50,ABV_close_MA_cl_200,ABV_3_TDUR_MA_cl_50,ABV_80_VDEV_10,filter_score,all_true
date,,,,,,,,
2023-12-06,False,False,False,False,True,False,1,False
2023-12-07,False,False,False,False,True,False,1,False
2023-12-08,False,False,False,False,True,False,1,False
2023-12-11,False,False,False,False,True,False,1,False
2023-12-12,False,False,False,False,True,False,1,False
...,...,...,...,...,...,...,...,...
2024-11-26,False,False,True,True,False,True,3,False
2024-11-27,False,False,True,True,False,True,3,False
2024-11-29,False,False,True,True,False,True,3,False


In [ ]:
ib.disconnect()

In [ ]:
from data import historical_data as hd
from ib_insync import *


tdata = hd.get_hist_data('TSLA', "52 weeksAgo", 'now', '1 day')
tdata

Loading data from data/historical_data_store/TSLA_1_day.csv
Stored data: 250 rows of data
Loading data from data/historical_data_store/TSLA_1_day.csv


,open,high,low,close,volume
date,,,,,
2023-12-06,240.61,246.57,238.85,240.37,1080540
2023-12-07,240.80,244.08,236.98,242.91,916952
2023-12-08,242.00,245.27,239.27,243.23,878583
2023-12-11,242.00,245.00,237.45,239.55,819699
2023-12-12,240.25,241.22,233.87,236.48,785221
...,...,...,...,...,...
2024-11-26,338.00,346.96,335.21,340.17,470824
2024-11-27,339.76,342.60,326.59,334.40,441879
2024-11-29,336.00,348.50,334.65,345.44,272292


In [ ]:
mansfield, roc, combined = analyze_sector(
    etf_symbol='XLE',
    lookback_period="26 weeksAgo",
    mansfield_period=150,
    roc_period=150,
    ma_short=20,
    ma_long=100,
    verbose=True
)

In [ ]:

    



# Example Usage
# Assuming `etf_data` and `market_data` are DataFrames with the required columns


from data import historical_data as hd
import compare



etf_data = hd.get_hist_data('XLF', "52 weeksAgo", 'now', '1 day')
market_data = hd.get_hist_data('SPY', "52 weeksAgo", 'now', '1 day')

analysis = compare.SectorAnalysis(etf_data, market_data)
analysis.compute_all(
    mansfield_period=200,  # Period for Mansfield RSI
    roc_period=200,         # Period for MA ROC ratio
    ma_short=50,           # Short MA period
    ma_long=200           # Long MA period
)
result_df = analysis.get_df()
today_metrics = analysis.get_today(verbose=True)
display(today_metrics)
result_df


Loading data from data/historical_data_store/XLF_1_day.csv
Stored data: 249 rows of data
Loading data from data/historical_data_store/XLF_1_day.csv
Loading data from data/historical_data_store/SPY_1_day.csv
Stored data: 250 rows of data
Loading data from data/historical_data_store/SPY_1_day.csv

=== Today's Market Analysis ===

Mansfield RSI: 4.25
MA ROC Ratio: 1.16
Combined Score: 2.71

📈 BULLISH


mansfield_rsi     4.254055
ma_roc_ratio      1.160618
combined_score    2.707337
dtype: float64

,etf_50_ma,etf_200_ma,market_50_ma,market_200_ma,mansfield_rsi,ma_roc_ratio,combined_score
date,,,,,,,
2024-09-25,43.9020,41.07935,551.0202,520.40360,-0.554947,0.000000,-0.277474
2024-09-26,43.9246,41.12345,551.3066,520.96265,-0.521474,-0.003399,-0.262437
2024-09-27,43.9616,41.16730,551.6480,521.50450,0.199075,0.125981,0.162528
2024-09-30,44.0124,41.21035,552.1318,522.04710,0.014426,0.025382,0.019904
2024-10-01,44.0514,41.24860,552.4014,522.52575,0.510626,0.061587,0.286106
2024-10-02,44.0928,41.28670,552.7696,523.01615,0.406175,-0.079116,0.163530
2024-10-03,44.1360,41.32415,553.2764,523.50465,0.084966,-0.144174,-0.029604
2024-10-04,44.1912,41.36600,553.9360,524.00725,1.017609,0.274168,0.645889
2024-10-07,44.2238,41.40340,554.4046,524.47240,0.609759,0.092630,0.351194


In [ ]:
etf_data

,open,high,low,close,volume
date,,,,,
2023-12-08,35.97,36.19,35.83,36.14,311722
2023-12-11,36.13,36.42,36.09,36.38,287274
2023-12-12,36.35,36.66,36.29,36.66,298369
2023-12-13,36.69,37.41,36.55,37.41,525548
2023-12-14,37.28,37.71,37.23,37.55,622257
...,...,...,...,...,...
2024-11-27,51.26,51.62,51.18,51.33,289386
2024-11-29,51.51,51.57,51.21,51.26,138172
2024-12-02,51.35,51.49,50.70,50.86,477889


In [ ]:
from ib_insync import IB, Stock
import xml.etree.ElementTree as ET

contract = Stock('TSLA', 'SMART', 'USD')

# fundamental_data = ib.reqFundamentalData(contract, 'ReportsFundamentals')
fundamental_data = ib.reqFundamentalData(contract, 'ReportSnapshot')
root = ET.fromstring(fundamental_data)

In [ ]:
def print_xml_tree(element, indent=0):
    """Print the XML tree structure with indentation"""
    # Print current element
    print('  ' * indent + f'<{element.tag}>', end='')
    
    # Print attributes if any
    if element.attrib:
        print(f' {element.attrib}', end='')
        
    # Print text content if it exists and isn't whitespace
    if element.text and element.text.strip():
        print(f' = {element.text.strip()}')
    else:
        print()
        
    # Print all children recursively
    for child in element:
        print_xml_tree(child, indent + 1)

print_xml_tree(root)

<ReportSnapshot> {'Major': '1', 'Minor': '0', 'Revision': '1'}
  <CoIDs>
    <CoID> {'Type': 'RepNo'} = C8279
    <CoID> {'Type': 'CompanyName'} = Tesla Inc
    <CoID> {'Type': 'IRSNo'} = 912197729
    <CoID> {'Type': 'CIKNo'} = 0001318605
    <CoID> {'Type': 'OrganizationPermID'} = 5088024644
  <Issues>
    <Issue> {'ID': '1', 'Type': 'C', 'Desc': 'Common Stock', 'Order': '1'}
      <IssueID> {'Type': 'Name'} = Ordinary Shares
      <IssueID> {'Type': 'Ticker'} = TSLA
      <IssueID> {'Type': 'RIC'} = TSLA.O
      <IssueID> {'Type': 'DisplayRIC'} = TSLA.OQ
      <IssueID> {'Type': 'InstrumentPI'} = 67910050
      <IssueID> {'Type': 'QuotePI'} = 72106022
      <Exchange> {'Code': 'NASD', 'Country': 'USA'} = NASDAQ
      <GlobalListingType> = OSR
      <MostRecentSplit> {'Date': '2022-08-25'} = 3.00003
    <Issue> {'ID': '2', 'Type': 'P', 'Desc': 'Preferred Stock', 'Order': '1'}
      <IssueID> {'Type': 'Name'} = Preference Shares Series B
      <IssueID> {'Type': 'InstrumentPI'} = 7447